In [1]:
import pandas as pd
import datetime as dt

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
path = "/content/gdrive/MyDrive/ML-Transportation-Rate/Raw_Data_Past_Four_Month"
MT_data_ori = pd.read_csv(path +'/Merged_DataFrame_hash_Version_addition.csv')
DAT_data = pd.read_excel(path +'/DAT_Rates_from_DB-9-21.xlsx')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (12,17) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
MT_data = MT_data_ori.sample(1000000)

# Create a column called ID_OR_OPTIONAL_FIELD, which join first three letter of ORIGIN zip
# and the first three letter of DEST zip and connect them by using "_"
MT_data['ID_OR_OPTIONAL_FIELD'] = MT_data['ORIGIN ZIP'].astype(str).str[:3] + '_' + MT_data['DEST ZIP'].astype(str).str[:3] 
MT_data['ID_OR_OPTIONAL_FIELD'] = MT_data['ID_OR_OPTIONAL_FIELD'].astype(str)


#Remove Columns of 'Unnamed: 0.1','Unnamed: 0'
MT_data = MT_data.drop(['Unnamed: 0.1','Unnamed: 0'], axis = 1)

# Drop the row if na is found in the MT_data['DISTANCE']
MT_data = MT_data[pd.notnull(MT_data['DISTANCE'])]

# Drop the row if na is found in the MT_data['WEIGHT]
MT_data = MT_data[pd.notnull(MT_data['WEIGHT'])]

# Drop the row if na is found in the MT_data['VOLUME']
MT_data = MT_data[pd.notnull(MT_data['VOLUME'])]


# Drop the row if na is found in the MT_data['ORIGIN CITY']
MT_data = MT_data[pd.notnull(MT_data['ORIGIN CITY'])]

# Only conisder shipment has Original zip code in USA and Canada
# Drop the row if ns is found in the MT_data['ORIGIN STATE']
MT_data = MT_data[pd.notnull(MT_data['ORIGIN STATE'])]
MT_data = MT_data.reset_index(drop=True)

US_list = [ 'AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA',
           'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME',
           'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM',
           'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
           'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY']
Canada_list = ['AB','BC','MB','NB','NL','NT','NS','NU','ON','PE','QC','SK','YT']
State_list = US_list + Canada_list

for i in range(len(MT_data)):
    if MT_data.loc[i,'ORIGIN STATE'] in State_list:
        pass
    else:
        MT_data = MT_data.drop(i)
        
        
# Drop the row if na is found in the MT_data['ORIGIN ZIP']
MT_data = MT_data[pd.notnull(MT_data['ORIGIN ZIP'])]
MT_data = MT_data.reset_index(drop=True)
# Change 4 digits zips to 5 digis
for i in (range(len(MT_data['ORIGIN ZIP']))):
    zipCode = str(MT_data.loc[i,'ORIGIN ZIP'])
    if zipCode.isdigit():
        if len(zipCode) < 5:
            MT_data.loc[i,'ORIGIN ZIP'] = "0" * (5 - len(zipCode)) + zipCode
    else:
        pass
            


# Actual mode will only consider LTL, and TL
# Drop the row if ns is found in the MT_data['ACTUAL MODE']
MT_data = MT_data[pd.notnull(MT_data['ACTUAL MODE'])]
MT_data['ACTUAL MODE'] = MT_data['ACTUAL MODE'].str.split('.').str[-1] 
MT_data['ACTUAL MODE'] = MT_data['ACTUAL MODE'].str.upper()
MT_data = MT_data.reset_index(drop=True)

acutual_mode_list = ['LTL','TL','INTERMODAL']

for i in range(len(MT_data)):
    if MT_data.loc[i,'ACTUAL MODE'] in acutual_mode_list:
        pass
    else:
      MT_data = MT_data.drop(i)
        
# Remove any nan cell in ACTUAL EQUIP
# Tokenize part information from the original cells
MT_data = MT_data[pd.notnull(MT_data['ACTUAL EQUIP'])]
MT_data['ACTUAL EQUIP'] = MT_data['ACTUAL EQUIP'].str.split('.').str[-1]

        
# Remove any nan cell in LINEHAUL COSTS
MT_data = MT_data[pd.notnull(MT_data['LINEHAUL COSTS'])]

# Remove any nan cell in FUEL COSTS
MT_data = MT_data[pd.notnull(MT_data['FUEL COSTS'])]

# Remove any nan cell in ACC. COSTS
MT_data = MT_data[pd.notnull(MT_data['ACC. COSTS'])]

# Remove any nan cell in total ACTUAL COST
MT_data = MT_data[pd.notnull(MT_data['TOTAL ACTUAL COST'])]

# Add a LINEHAUL UNIT COSTS to the dataframe
MT_data['LINEHAUL UNIT COSTS'] = MT_data['LINEHAUL COSTS']/MT_data['DISTANCE']

# Drop the row if na is found in the MT_data['DEST CITY']
MT_data = MT_data[pd.notnull(MT_data['DEST CITY'])]

# Only conisder shipment has dest zip code in USA and Canada
# Drop the row if ns is found in the MT_data['ORIGIN STATE']
MT_data = MT_data[pd.notnull(MT_data['DEST STATE'])]
MT_data = MT_data.reset_index(drop=True)

for i in range(len(MT_data)):
    if MT_data.loc[i,'DEST STATE'] in State_list:
        pass
    else:
        MT_data = MT_data.drop(i)
        
# Drop the row if na is found in the MT_data['DEST ZIP']
MT_data = MT_data[pd.notnull(MT_data['DEST ZIP'])]
MT_data = MT_data.reset_index(drop=True)
# Change 4 digits zips to 5 digis
for i in (range(len(MT_data['DEST ZIP']))):
    zipCode = str(MT_data.loc[i,'DEST ZIP'])
    if zipCode.isdigit():
        if len(zipCode) < 5:
            MT_data.loc[i,'DEST ZIP'] = "0" * (5 - len(zipCode)) + zipCode
    else:
        pass

# Clean the column of PU_APPT as a new column called File_paried_data
MT_data['File_paried_data'] = pd.to_datetime(MT_data['PU_APPT'].str.split(' ').str[0], errors = 'coerce')
MT_data['File_paried_data'] = MT_data['File_paried_data'].dt.strftime('%Y-%m-%d')

In [ ]:
#DATA_data Cleaning

# Set ID_OR_OPTIONAL_FIELD as an objective column
DAT_data['ID_OR_OPTIONAL_FIELD'] = DAT_data['ID_OR_OPTIONAL_FIELD'].astype(str)

# Clean the column of File_data  and add year '2022' before the month
DAT_data['File_data'] = DAT_data['SOURCE_FILE_NAME'].str.split(' ').str[-1]
DAT_data['File_data'] = DAT_data['File_data'].str.split('.').str[0]
DAT_data['File_data'] = '2022-' + DAT_data['File_data']
DAT_data['File_data'] = pd.to_datetime(DAT_data['File_data'])

# Remove columns have ALL na
DAT_data = DAT_data.drop('NOTE',axis = 1)


# Drop the row if na is found in the DAT_data['SPOT_AVG_LINEHAUL_RATE']
DAT_data = DAT_data[pd.notnull(DAT_data['SPOT_AVG_LINEHAUL_RATE'])]

# Drop the row if na is found in the DAT_data['SPOT_TIME_FRAME']
DAT_data = DAT_data[pd.notnull(DAT_data['SPOT_TIME_FRAME'])]

# Drop the row if na is found in the DAT_data['CONTRACT_AVG_LINEHAUL_RATE']
DAT_data = DAT_data[pd.notnull(DAT_data['CONTRACT_AVG_LINEHAUL_RATE'])]

# Drop the row if na is found in the DAT_data['CONTRACT_TIME_FRAME']
DAT_data = DAT_data[pd.notnull(DAT_data['CONTRACT_TIME_FRAME'])]

In [ ]:
# Join two tables
#Join the table by setting ID_OR_OPTIONAL_FIELD as the key
merge_df = MT_data.merge(DAT_data, on = 'ID_OR_OPTIONAL_FIELD', how = 'left')

In [ ]:
# Data Cleaning
# 1. Remove if File_data is NA
merge_df_remove_NA = merge_df
merge_df_remove_NA = merge_df_remove_NA[pd.notnull(merge_df_remove_NA['File_data'])] 
merge_df_remove_NA = merge_df_remove_NA.reset_index(drop=True)


# 2. Only keeps row if File_paried_data is not larger than File_data
#for i in range(len(merge_df)):
#    if pd.to_datetime(merge_df.loc[i, 'File_paried_data']) <= pd.to_datetime(merge_df.loc[i, 'File_data']):
#        merge_df = merge_df.drop(index=i)

for i in range(len(merge_df_remove_NA)):
    if pd.to_datetime(merge_df_remove_NA.loc[i, 'File_paried_data']) <= pd.to_datetime(merge_df_remove_NA.loc[i, 'File_data']):
        merge_df_remove_NA = merge_df_remove_NA.drop(index=i)


In [ ]:
#Show all columns in the file
pd.set_option('display.max_columns', None)
merge_df_remove_NA.head()

In [ ]:
# Review column name
merge_df_remove_NA.columns

In [ ]:
#Selected Input columns
required_column = ['SHIPMENT ID', 'CUSTOMER', 'DISTANCE', 'CASES', 'WEIGHT', 'VOLUME',
       'SOURCE LOCATION ID', 'ORIGIN NAME', 'ORIGIN CITY', 'ORIGIN STATE',
       'ORIGIN ZIP', 'DEST LOCATION ID', 'CONSIGNEE NAME', 'DEST CITY',
       'DEST STATE', 'DEST ZIP', 'ACTUAL CARRIER', 'ACTUAL MODE',
       'ACTUAL EQUIP', 'LINEHAUL COSTS','TOTAL ACTUAL COST', 'PU_APPT','DL_APPT',
       'LINEHAUL UNIT COSTS','PC_MILER_PRACTICAL_MILEAGE',
       'SPOT_AVG_LINEHAUL_RATE', 'SPOT_LOW_LINEHAUL_RATE',
       'SPOT_HIGH_LINEHAUL_RATE', 'SPOT_FUEL_SURCHARGE', 'SPOT_TIME_FRAME',
       'SPOT_ORIGIN_GEO_EXPANSION', 'SPOT_DESTINATION_GEO_EXPANSION',
       'SPOT_NUMBER_OF_COMPANIES', 'SPOT_NUMBER_OF_REPORTS',
       'SPOT_LINEHAUL_RATE_STDDEV', 'SPOT_YOUR_OWN_AVG_LINEHAUL_RATE',
       'SPOT_YOUR_OWN_NUMBER_OF_REPORTS', 'SPOT_ERROR',
       'CONTRACT_AVG_LINEHAUL_RATE', 'CONTRACT_LOW_LINEHAUL_RATE',
       'CONTRACT_HIGH_LINEHAUL_RATE', 'CONTRACT_FUEL_SURCHARGE',
       'CONTRACT_AVG_ACCESSORIAL_EXCLUDES_FUEL', 'CONTRACT_TIME_FRAME',
       'CONTRACT_ORIGIN_GEO_EXPANSION', 'CONTRACT_DESTINATION_GEO_EXPANSION',
       'CONTRACT_NUMBER_OF_COMPANIES', 'CONTRACT_NUMBER_OF_REPORTS',
       'CONTRACT_LINEHAUL_RATE_STDDEV', 'CONTRACT_YOUR_OWN_AVG_LINEHAUL_RATE',
       'CONTRACT_YOUR_OWN_NUMBER_OF_REPORTS', 'CONTRACT_ERROR', 'ORIG_CITY',
       'ORIG_STATE', 'ORIG_POSTAL_CODE', 'DEST_CITY', 'DEST_STATE',
       'DEST_POSTAL_CODE', 'TRUCK_TYPE', 'RUN_ID', 'SOURCE_FILE_NAME']
cleaned_df = merge_df_remove_NA[required_column]

In [ ]:
cleaned_df.head()

# Create 3 seperate sample datasets for TL, LTL, and railway

In [ ]:
LTL_Cleaned_Data = cleaned_df[cleaned_df['ACTUAL MODE'] == 'LTL']
TL_Cleaned_Data = cleaned_df[cleaned_df['ACTUAL MODE'] == 'TL']
INTERMODAL_Cleaned_Data = cleaned_df[cleaned_df['ACTUAL MODE'] == 'INTERMODAL']

# Output Dataset

In [ ]:
Output_path = "/content/gdrive/MyDrive/ML-Transportation-Rate/sample data"
cleaned_df.to_csv( Output_path +'/Overall_Sample_Data.csv')
LTL_Cleaned_Data.to_csv( Output_path +'/LTL_Cleaned_Sample_Data.csv')
TL_Cleaned_Data.to_csv( Output_path +'/TL_Cleaned_Sample_Data.csv')
INTERMODAL_Cleaned_Data.to_csv( Output_path +'/INTERMODAL_Cleaned_Sample_Data.csv')